## 1.查看Paddle、PaddleHub版本及模型信息

In [46]:
import paddle
import paddlehub as h
from paddlehub.datasets.base_nlp_dataset import TextClassificationDataset

# 打印paddle的版本信息
print('-----打印paddle的版本信息:-----')
print(paddle.__version__)

# 打印paddlehub的版本信息
print('-----打印paddlehub的版本信息:-----')
print(h.__version__)

# 打印ernie模型信息
print('-----打印ernie模型信息:-----')
print(h.Module(name='ernie'))

-----打印paddle的版本信息:-----
2.0.2
-----打印paddlehub的版本信息:-----
2.0.4
-----打印ernie模型信息:-----


[2021-04-18 16:24:30,445] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/ernie-1.0/ernie_v1_chn_base.pdparams


Ernie(
  (model): ErnieModel(
    (embeddings): ErnieEmbeddings(
      (word_embeddings): Embedding(18000, 768, padding_idx=0, sparse=False)
      (position_embeddings): Embedding(513, 768, sparse=False)
      (token_type_embeddings): Embedding(2, 768, sparse=False)
      (layer_norm): LayerNorm(normalized_shape=[768], epsilon=1e-05)
      (dropout): Dropout(p=0.1, axis=None, mode=upscale_in_train)
    )
    (encoder): TransformerEncoder(
      (layers): LayerList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiHeadAttention(
            (q_proj): Linear(in_features=768, out_features=768, dtype=float32)
            (k_proj): Linear(in_features=768, out_features=768, dtype=float32)
            (v_proj): Linear(in_features=768, out_features=768, dtype=float32)
            (out_proj): Linear(in_features=768, out_features=768, dtype=float32)
          )
          (linear1): Linear(in_features=768, out_features=3072, dtype=float32)
          (dropout): Dropout(p=0, axis=N

## 2.打印THUCNews验证集前3条、后3条数据  

In [ ]:
! tar --help

### 2.1 解压数据

In [ ]:
! tar -xvf data/data80226/thu_news.tar

thu_news/
thu_news/test.txt
thu_news/valid.txt
thu_news/train.txt


### 2.2打印验证集前三条数据、后三条数据

In [45]:
# 前三条数据
!head -n 3 thu_news/valid.txt
# 后三条数据
!tail -n 3 thu_news/valid.txt

text_a	label
直击特里勾手助小牛反超神鸟发威火箭仍处劣势　　新浪体育讯　北京时间4月16日消息，火箭今天迎来常规赛的收官战。客场挑战达拉斯小牛的比赛将关系到火箭队最终的排名，目前西部的竞争仍然非常激励。尤其是西部第二到第七的六支球队将在今天展开捉对厮杀的情况下，黄蜂、小牛、开拓者以及马刺都有可能是火箭队的下一个对手。以下为本场比赛的精彩瞬间——　　第四节比赛还剩下8分多钟，姚明重新回到场上，但是小牛队的霍里随后在右翼接队友传球后上演一记三分跳投，虽然身体动作已经变形，但是他仍然将球命中，随后基德在弧顶处的一记三分跳投帮助小牛终于将比分反超，火箭在进攻中直接打不开局面，而小牛更是利用特里空切后的一记勾手将比分反超至4分，包括库班在内的所有小牛现场球迷都站起来振臂高呼，而此时阿德尔曼还在场边低头深沉的思索中。　　比赛还剩下最后5分多钟，洛瑞带球突破中被小牛队员犯规，洛瑞来不及刹车让自己直接撞到了观众席上，右腿疼痛难忍的他脸都几乎变形，结果他两罚两中，火箭队还落后两分。随后，洛瑞将球直接传给此时已经回到场上的阿泰，野兽此时将球稳下，并没有急于进攻，随后他顺势将球塞给兰德里，神鸟利用对方站位空隙直接杀到篮下，上篮成功，火箭将比分终于追至80平。场上局面对于火箭来说绝对是不容有失。　　(sabrina)	体育
北京网购年消费额112亿元　　商报讯(记者吴文治)昨日，淘宝网发布的《2009-2010年度中国网购热门城市报告》显示，北京年度网购消费力达112.5亿元，与上海相差近62亿元，位列十大热门消费力城市第二位。此外，男性网购的消费金额高出女性，与“女性是网购主力军”的传统观念不符。　　淘宝公布的报告显示，中国网购消费力十大城市分别是上海、北京、深圳、杭州、广州、南京、苏州、天津、温州和宁波。主要集中在以江浙沪为主的长三角地区、以广深为主的珠三角地区和以北京为主的京津地区。北京年度网购112.5亿元的消费额，占国内城市网购消费额的5.6%。　　中国网购消费力十大城市的消费金额性别来源比例中，男性占比超过了女性。前者占比达到53.5%，后者则为46.5%。不过，在成交人数、成交笔数等关键数据上显示，女性消费者均高于男性。此外，在十大网购热门城市中，25岁-34岁的群体成为网购消费的主力军，占比达到62.49%。	科技
事业测试：你工作易受他人干扰吗(图)　　独家撰

## 3.基于PaddleHub的新闻文本分类任务

利用PaddleHub的预训练模型ernie完成新闻分类的迁移学习

### 3.1 自定义数据集

In [ ]:
class ThuNews(TextClassificationDataset):
    #数据集的标签列表
    
    def __init__(self, tokenizer, max_seq_len=128,mode='train'):
        if mode == 'train':
            data_file = 'train.txt'
        elif mode == 'test':
            data_file = 'test.txt'
        else:
            data_file = 'valid.txt'

        super(ThuNews, self).__init__(
            base_path='thu_news',
            data_file=data_file,
            tokenizer=tokenizer,
            max_seq_len=max_seq_len,
            mode=mode,
            is_file_with_header=True,
            label_list=['体育', '科技', '社会', '娱乐', '股票', '房产', '教育', '时政', '财经', '星座', '游戏', '家居', '彩票', '时尚']


### 3.2 模型加载

In [21]:
# 模型加载
model = h.Module(name='ernie', task='seq-cls', num_classes=14)
tokenizer = model.get_tokenizer()

[2021-04-18 15:57:57,504] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/ernie-1.0/ernie_v1_chn_base.pdparams
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/layers.py:1303: UserWarning: Skip loading for classifier.weight. classifier.weight is not found in the provided dict.
  warnings.warn(("Skip loading for {}. ".format(key) + str(err)))
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/layers.py:1303: UserWarning: Skip loading for classifier.bias. classifier.bias is not found in the provided dict.
  warnings.warn(("Skip loading for {}. ".format(key) + str(err)))
[2021-04-18 15:57:59,227] [    INFO] - Found /home/aistudio/.paddlenlp/models/ernie-1.0/vocab.txt


In [22]:
# 实例化数据集
train_dataset = ThuNews(tokenizer, mode='train')
dev_dataset = ThuNews(tokenizer, mode='dev')

### 3.3 加载数据、模型训练 

In [23]:
optimizer = paddle.optimizer.Adam(learning_rate=5e-5, parameters=model.parameters())
trainer = h.Trainer(model, optimizer, checkpoint_dir='ckpt', use_gpu=True)
trainer.train(train_dataset, epochs=5, batch_size=32, eval_dataset=dev_dataset, save_interval=1)

[2021-04-18 16:00:03,928] [ WARNING] - PaddleHub model checkpoint not found, start from scratch...
[2021-04-18 16:00:06,249] [   TRAIN] - Epoch=1/5, Step=10/282 loss=2.2596 acc=0.2938 lr=0.000050 step/sec=4.33 | ETA 00:05:25
[2021-04-18 16:00:08,360] [   TRAIN] - Epoch=1/5, Step=20/282 loss=1.4753 acc=0.6344 lr=0.000050 step/sec=4.74 | ETA 00:05:11
[2021-04-18 16:00:10,473] [   TRAIN] - Epoch=1/5, Step=30/282 loss=1.0994 acc=0.7188 lr=0.000050 step/sec=4.73 | ETA 00:05:07
[2021-04-18 16:00:12,580] [   TRAIN] - Epoch=1/5, Step=40/282 loss=0.7771 acc=0.8187 lr=0.000050 step/sec=4.75 | ETA 00:05:04
[2021-04-18 16:00:14,691] [   TRAIN] - Epoch=1/5, Step=50/282 loss=0.6346 acc=0.8594 lr=0.000050 step/sec=4.74 | ETA 00:05:03
[2021-04-18 16:00:16,829] [   TRAIN] - Epoch=1/5, Step=60/282 loss=0.6976 acc=0.8187 lr=0.000050 step/sec=4.68 | ETA 00:05:02
[2021-04-18 16:00:18,967] [   TRAIN] - Epoch=1/5, Step=70/282 loss=0.5340 acc=0.8812 lr=0.000050 step/sec=4.68 | ETA 00:05:02
[2021-04-18 16:00:2

### 3.4 模型预测 

In [26]:
# 在测试数据集中随机挑选出一条数据，标签为【房产】
data = [["二环小户型前门前总价140万全款98折(图)　　新浪房产讯(编辑晏阳)　前门前(论坛相册户型样板间地图搜索)2009年4月25日开盘，均价25800元/平方米。项目分三期开发，一期为高档住宅，进深仅9.7米，南北通透，容积率2.3，目前在售为1、2、3号楼，6层板楼2梯4户，户型面积为60平米-170平米，均价25800元/平米，总价140万/套起，2009年11月30日入住。一次性98折！　　前门前项目位于二环内前门商业街的东南角，向南步行300米就是举世闻名的天坛公园，向东500米即为繁华的新世界商圈，总建筑面积12.5万平方米，占地面积3.9万平米。项目由西向东分为ABC三个区，A区是商务公寓，B区为50—170?阳光纯板，南北通透。C区是一个三进式仿古四合院。B区住宅即将开盘。　　前门前地处二环核心，前观天安门，后揽天坛公园，三线地铁交汇，距地铁7号线大都市街站约500米。周边遍布老北京五大商圈、同仁、协和医院等众多生活便利。项目青花瓷电梯间立基与天安门、祈年殿一线。规划上，采用中式围合布局，与项目周边众多古建和谐生趣，每个组团都有各自的院落，同时通过挑空一层，使得每个组团有益连接，种植更多大树、水系。内部空间上，楼体坐南朝北，室内空间格局方正，户户朝阳。　　以上信息仅供参考，最终以开发商公布为准。　　点击查看更多楼盘开盘信息。"]]

label_list=['体育', '科技', '社会', '娱乐', '股票', '房产', '教育', '时政', '财经', '星座', '游戏', '家居', '彩票', '时尚']
label_map = { 
    idx: label_text for idx, label_text in enumerate(label_list)
}

model = h.Module(name='ernie',task='seq-cls',load_checkpoint='ckpt/epoch_5/model.pdparams',label_map=label_map)
results = model.predict(data, max_seq_len=128, batch_size=1, use_gpu=True)
for idx, text in enumerate(data):
    print('Data: {} \t Lable: {}'.format(text[0], results[idx]))

[2021-04-18 16:09:27,412] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/ernie-1.0/ernie_v1_chn_base.pdparams
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/layers.py:1303: UserWarning: Skip loading for classifier.weight. classifier.weight is not found in the provided dict.
  warnings.warn(("Skip loading for {}. ".format(key) + str(err)))
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/layers.py:1303: UserWarning: Skip loading for classifier.bias. classifier.bias is not found in the provided dict.
  warnings.warn(("Skip loading for {}. ".format(key) + str(err)))
[2021-04-18 16:09:32,603] [    INFO] - Loaded parameters from /home/aistudio/ckpt/epoch_5/model.pdparams
[2021-04-18 16:09:32,741] [    INFO] - Found /home/aistudio/.paddlenlp/models/ernie-1.0/vocab.txt


Data: 二环小户型前门前总价140万全款98折(图)　　新浪房产讯(编辑晏阳)　前门前(论坛相册户型样板间地图搜索)2009年4月25日开盘，均价25800元/平方米。项目分三期开发，一期为高档住宅，进深仅9.7米，南北通透，容积率2.3，目前在售为1、2、3号楼，6层板楼2梯4户，户型面积为60平米-170平米，均价25800元/平米，总价140万/套起，2009年11月30日入住。一次性98折！　　前门前项目位于二环内前门商业街的东南角，向南步行300米就是举世闻名的天坛公园，向东500米即为繁华的新世界商圈，总建筑面积12.5万平方米，占地面积3.9万平米。项目由西向东分为ABC三个区，A区是商务公寓，B区为50—170?阳光纯板，南北通透。C区是一个三进式仿古四合院。B区住宅即将开盘。　　前门前地处二环核心，前观天安门，后揽天坛公园，三线地铁交汇，距地铁7号线大都市街站约500米。周边遍布老北京五大商圈、同仁、协和医院等众多生活便利。项目青花瓷电梯间立基与天安门、祈年殿一线。规划上，采用中式围合布局，与项目周边众多古建和谐生趣，每个组团都有各自的院落，同时通过挑空一层，使得每个组团有益连接，种植更多大树、水系。内部空间上，楼体坐南朝北，室内空间格局方正，户户朝阳。　　以上信息仅供参考，最终以开发商公布为准。　　点击查看更多楼盘开盘信息。 	 Lable: 房产


请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 